# Dashboard Explicativo de Clustering de Clientes
Incluye KPIs, EDA visual, clustering, interpretación de tópicos, callbacks y paneles intuitivos. Ajustado para evitar errores de formato.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA, NMF
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
import plotly.express as px
import plotly.graph_objects as go
import dash
from dash import dcc, html, Input, Output, dash_table
import dash_bootstrap_components as dbc
from scipy.stats import entropy
import datetime as dt
import warnings
warnings.filterwarnings('ignore')

## 1. Carga de datos y EDA básico

In [ ]:
file_path = "Data_Set_Global.xlsx"
pedidos = pd.read_excel(file_path, sheet_name="Pedidos")
pedidos["total_price"] = pedidos["quantity"] * pedidos["unit_price"]
pedidos["order_date"] = pd.to_datetime(pedidos["order_date"])
pedidos["promised_delivery_time"] = pd.to_datetime(pedidos["promised_delivery_time"])
pedidos["actual_delivery_time"] = pd.to_datetime(pedidos["actual_delivery_time"])
pedidos["delivery_delay_min"] = (
    (pedidos["actual_delivery_time"] - pedidos["promised_delivery_time"]).dt.total_seconds() / 60
)
pedidos = pedidos.dropna(subset=["order_id", "customer_id", "order_date"]).copy()

## 2. KPIs generales

In [ ]:
kpi_dict = {
    "Total Clientes": pedidos["customer_id"].nunique(),
    "Total Órdenes": pedidos["order_id"].nunique(),
    "Total Productos": pedidos["product_id"].nunique(),
    "Facturación Total": pedidos["total_price"].sum(),
    "Periodos Analizados": f"{pedidos['order_date'].min().date()} a {pedidos['order_date'].max().date()}"
}

## 3. EDA visual y descriptivo

In [ ]:
# Distribución de órdenes por segmento
df_seg = pedidos.groupby("customer_segment")["order_id"].count().reset_index()
fig_bar_segment = px.bar(df_seg, x="customer_segment", y="order_id", text="order_id",
                        labels={"order_id": "# Órdenes"}, title="Órdenes por Segmento de Cliente")

# Top 5 productos por segmento
N = 5
topN = (
    pedidos.groupby(["customer_segment", "product_name"])
    .size().groupby(level=0, group_keys=False).nlargest(N)
    .reset_index(name="compras")
)
top5_prod_segment = {}
for seg in topN["customer_segment"].unique():
    top5_prod_segment[seg] = topN[topN["customer_segment"] == seg][["product_name", "compras"]].to_dict('records')

# Correlación de numéricas
num_cols = pedidos.select_dtypes("number").columns
corr = pedidos[num_cols].corr()
fig_corr = px.imshow(corr, color_continuous_scale='RdBu', title="Matriz de Correlación Variables Numéricas")

# Histograma de demoras
fig_delay = px.histogram(pedidos, x="delivery_delay_min", nbins=50, title="Demora de entrega (min)")

# Conversión: días a primera compra
agg = (
    pedidos.groupby("customer_id")
    .agg(
        first_order=("order_date", "min"),
        registration_date=("registration_date", "first")
    ).reset_index()
)
agg["days_to_first"] = (agg["first_order"] - agg["registration_date"]).dt.days
fig_conversion = px.histogram(agg, x="days_to_first", nbins=30, title="Días hasta primera compra")

## 4. Variables para clustering

In [ ]:
# RFM
snapshot_date = pedidos["order_date"].max() + dt.timedelta(days=1)
rfm = pedidos.groupby("customer_id").agg(
    recency=("order_date", lambda date: (snapshot_date - date.max()).days),
    frequency=("order_id", "nunique"),
    monetary=("total_price", "sum")
)
scaler_rfm = StandardScaler()
rfm_z = pd.DataFrame(scaler_rfm.fit_transform(rfm), index=rfm.index, columns=[c+"_z" for c in rfm.columns])

# Entropía de pago
pay_counts = pedidos.groupby(["customer_id", "payment_method"]).size().unstack(fill_value=0)
pay_probs = pay_counts.div(pay_counts.sum(axis=1), axis=0)
payment_entropy = pay_probs.apply(lambda row: entropy(row, base=2), axis=1)
customer_payment_diversity = payment_entropy.rename("payment_entropy").reset_index()

# NMF tópicos cliente-producto
cust_prod = pedidos.groupby(["customer_id", "product_id"]).size().unstack(fill_value=0)
K = 10
nmf = NMF(n_components=K, init="random", random_state=42)
W = nmf.fit_transform(cust_prod)
H = nmf.components_
customer_topics = pd.DataFrame(W, index=cust_prod.index, columns=[f"topic_{i+1}" for i in range(K)])

# Matriz final para clustering
full_df = (
    rfm_z
    .join(customer_payment_diversity.set_index("customer_id"))
    .join(customer_topics)
).fillna(0)
X_scaled = StandardScaler().fit_transform(full_df.select_dtypes(include=np.number))

## 5. Clustering final

In [ ]:
K_OPT = 3
kmeans = KMeans(n_clusters=K_OPT, random_state=42, n_init=10)
cluster_labels = kmeans.fit_predict(X_scaled)
full_df["cluster"] = cluster_labels
full_df.index.name = "customer_id"
profile_cols = [c for c in full_df.columns if c.startswith(('recency','frequency','monetary','payment_entropy','topic_'))]
cluster_profile = (
    full_df.groupby("cluster")[profile_cols]
    .mean().round(2)
)
cluster_size = full_df["cluster"].value_counts().sort_index()

## 6. Interpretación de tópicos

In [ ]:
product_ids = cust_prod.columns
prod_meta = pd.read_excel(file_path, sheet_name="Productos").set_index("product_id")
top_words_topic = {}
for i, topic in enumerate(H):
    top_ids = np.argsort(topic)[-5:][::-1]
    top_names = prod_meta.loc[product_ids[top_ids], "product_name"].tolist()
    top_words_topic[f"topic_{i+1}"] = top_names

## 7. Dashboard interactivo

In [ ]:
app = dash.Dash(__name__, external_stylesheets=[dbc.themes.MINTY])
app.layout = dbc.Container([
    html.H2("Dashboard de Segmentación de Clientes", style={"marginTop": 15}),
    html.Hr(),
    dbc.Row([
        dbc.Col(
            dbc.Card([
                dbc.CardHeader(k),
                dbc.CardBody(html.H5(f"{v:,}" if isinstance(v, (int, float)) else str(v)))
            ]), width=3
        ) for k, v in kpi_dict.items()
    ]),
    html.Br(),
    html.H3("Análisis Exploratorio de los Datos (EDA)"),
    html.P("""Se exploran patrones de compra, productos más vendidos, correlaciones y tiempos clave para contextualizar la segmentación posterior."""),
    dbc.Row([
        dbc.Col([dcc.Graph(figure=fig_bar_segment)], width=6),
        dbc.Col([dcc.Graph(figure=fig_corr)], width=6),
    ]),
    dbc.Row([
        dbc.Col([dcc.Graph(figure=fig_delay)], width=6),
        dbc.Col([dcc.Graph(figure=fig_conversion)], width=6),
    ]),
    html.Br(),
    html.H5("Top 5 productos por segmento de cliente"),
    html.Ul([
        html.Li([
            html.B(f"Segmento {seg}: "), 
            ', '.join([f"{d['product_name']} ({d['compras']})" for d in lst])
        ]) for seg, lst in top5_prod_segment.items()
    ]),
    html.Hr(),
    html.H3("Clustering & Segmentación de Clientes"),
    html.P("""Se aplicó KMeans sobre variables RFM, entropía y tópicos NMF. El número óptimo de clusters es 3, balanceando interpretabilidad y robustez estadística. El gráfico PCA muestra la separación de clusters en 2D."""),
    dcc.Graph(
        figure=px.scatter(
            x=PCA(n_components=2).fit_transform(X_scaled)[:,0],
            y=PCA(n_components=2).fit_transform(X_scaled)[:,1],
            color=full_df["cluster"].astype(str),
            title="Clusters en espacio PCA (2D)",
            labels={"x": "PCA1", "y": "PCA2", "color": "Cluster"},
            hover_data={"customer_id": full_df.index}
        )
    ),
    html.H5("Tamaño de cada cluster"),
    dcc.Graph(figure=px.bar(x=cluster_size.index.astype(str), y=cluster_size.values, labels={'x':'Cluster','y':'#Clientes'})),
    html.Hr(),
    html.H4("Resumen descriptivo de clusters"),
    html.P("""La tabla muestra los valores promedio de cada variable (RFM, entropía y tópicos) en cada cluster. Así puedes ver qué distingue a cada segmento."""),
    dash_table.DataTable(
        columns=[{"name": i, "id": i} for i in cluster_profile.columns],
        data=cluster_profile.reset_index().to_dict('records'),
        style_table={'width': '100%'}, style_cell={'textAlign': 'center'},
        page_size=10
    ),
    html.Br(),
    html.H4("Distribución de tópicos en clusters (boxplot interactivo)"),
    html.P("""Selecciona un tópico para explorar su distribución por cluster. Los tópicos resumen patrones de consumo comunes extraídos automáticamente."""),
    dcc.Dropdown(
        id='topic-dropdown',
        options=[{"label": t, "value": t} for t in customer_topics.columns],
        value=customer_topics.columns[0],
        clearable=False
    ),
    dcc.Graph(id='topic-box'),
    html.Hr(),
    html.H4("Interpretación de tópicos NMF"),
    html.P("""Cada tópico representa un patrón de consumo distinto. La tabla muestra los productos más representativos de cada tópico."""),
    dash_table.DataTable(
        columns=[{"name": k, "id": k} for k in top_words_topic.keys()],
        data=[{k: ', '.join(v) for k, v in top_words_topic.items()}],
        style_table={'width': '100%'}, style_cell={'textAlign': 'left'},
    ),
    html.Hr(),
    html.H4("Detalle de clientes (filtrable por cluster)"),
    dcc.Dropdown(
        id='dropdown-cluster',
        options=[{"label": str(c), "value": str(c)} for c in sorted(full_df['cluster'].unique())],
        value=None, clearable=True, placeholder='Filtrar por cluster...'
    ),
    dash_table.DataTable(
        id='clientes-table',
        columns=[{"name": i, "id": i} for i in ["recency_z","frequency_z","monetary_z","payment_entropy"]+list(customer_topics.columns)+["cluster"]],
        data=full_df.reset_index().to_dict('records'),
        page_size=12, filter_action="native", sort_action="native", style_table={'overflowX': 'auto'}, style_cell={'textAlign': 'center'}
    ),
], fluid=True)

# Callbacks interactivos para boxplot de tópicos y filtrado de clientes
@app.callback(
    Output('topic-box', 'figure'),
    Input('topic-dropdown', 'value')
)
def update_topic_box(topic):
    fig = px.box(full_df.reset_index(), x=full_df['cluster'].astype(str), y=topic, points='all',
                title=f"Boxplot de {topic} por cluster", labels={"x": "Cluster", "y": topic})
    return fig

@app.callback(
    Output('clientes-table', 'data'),
    Input('dropdown-cluster', 'value')
)
def update_table(cluster_val):
    df = full_df.reset_index()
    if cluster_val is not None:
        df = df[df['cluster'] == int(cluster_val)]
    return df[["recency_z","frequency_z","monetary_z","payment_entropy"]+list(customer_topics.columns)+["cluster"]].to_dict('records')

# Para lanzar el dashboard, descomenta y ejecuta:
# app.run(debug=True, port=8051)
# Y abre: http://127.0.0.1:8051/